# National Model 1a

- Equifax, TU, Experian National model 
- Features selection on national data
- Time split on Equifax

The notebook covers all three steps for creatign a new national model; feature selection, validation model and fold-in model. depending on the applied use case, not each step may be necessary (e.g., if the feature selection does not change compaed to the previous interation, step 1 may be skipped, etc.

In [1]:
import pandas as pd
import numpy as np

import model_engine
model_engine.__version__

'v1.13.0'

In [2]:
from model_engine.power.post_sale import NationalModelBuilder
from model_engine.io.loaders import load_json

import json

/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/zamlexplain/utils_compile.py:23: NumbaPendingDeprecationWarning: The 'pycc' module is pending deprecation. Replacement technology is being developed.

Pending Deprecation in Numba 0.57.0. For more information please see: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-the-numba-pycc-module
  from numba.pycc import CC
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/shap/utils/_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
/home/lyt/.conda/envs/penfed_modeling/lib/python3.1

In [3]:
import os
import numba as nb
n_jobs = os.getenv('CPU_LIMIT', default=-1)
n_jobs = 20
print(n_jobs)
nb.set_num_threads(n_jobs)

20


In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
input_client_name = 'penfed'
input_project_name = 'autoloan'
input_bureau_name = 'normalized'
model_type = 'standard_model_member_data' # options are 'standard_model', 'mega_model', 'standard_model_member_data', 'standard_model_ln_data', ''- None defaults to 'standard_model'

print(input_client_name, input_project_name, input_bureau_name, model_type)

# national_model_id = 'nationalmodel1b'

penfed autoloan normalized standard_model_member_data


## Create a national model

In [6]:
# client' feature selection model

feature_selection_model_id = 'nationalmodel1a'

In [7]:
feature_selection_model_output_path = f'/d/shared/users/lyt/penfed_poc1_national_models/{feature_selection_model_id}'
feature_selection_model_output_path

'/d/shared/users/lyt/penfed_poc1_national_models/nationalmodel1a'

In [8]:
config = {
  "model_type": "standard_model",
  "storage_location": "s3",
  "lookalike": {
    "bureau": "experian",
    "accounts":[
      "auto"
    ],
    "splits": [
      "train",
      "valid",
      "test"
    ],
    "pull_names": None,
    "states": None,
    "max_row": 1000,
    "tables": ['trade', 'inq', 'bnkr', 'collec', 'target', 'app', 'member'],
  },
  "model_config": {
      "data_split": {
          "train": {
            "start_date": "2016-04-01",
            "end_date": "2021-04-01"
          },
          "valid": {
            "start_date": "2021-04-01",
            "end_date": "2021-07-01"
          },
          "test": {
            "start_date": "2021-07-01",
            "end_date": "2022-01-01"
          }
    },
    "target": "final_DQ60_m24",
    "base_features": f'{feature_selection_model_output_path}/keep_features.json',
#    "feature_filter_rules": "/home/lyt/CODEBASE/model-engine_penfed/model-engine/model_engine/assets/power/feature_rules/intuitive_features.json",
    "memory_efficient": True,
    "pipeline_factory": None,
    "bivariate_fe_instructions": [],
    "monotonic_constraints_list": [],
    "exclusion_list": [],
    "feature_definition_list": [],
    "key_factor_mapping_list": [],
    "fold_valid": True,
    "mandatory_features": []
  }
}

In [9]:
national_model_id = 'nationalmodel1b_exp_test'
national_model_output_path = f'/d/shared/users/lyt/penfed_poc1_national_models/{national_model_id}'


In [10]:
national_model_output_path

'/d/shared/users/lyt/penfed_poc1_national_models/nationalmodel1b_exp_test'

In [11]:
nmb = NationalModelBuilder(
    configuration=config,
    model_output_path=national_model_output_path,
    overwrite_model_output=True
)
nmb.run()

Attempting to create a national model with full data split here /d/shared/users/lyt/penfed_poc1_national_models/nationalmodel1b_exp_test/supporting_model
Sampling experian, national_1.3
Sampling experian, national_2
Sampling experian, national_1.3
Sampling experian, national_2
Sampling experian, national_1.3
Sampling experian, national_2



/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/lyt/.conda/envs/penfed_modeling/lib/python3.10/site-packages/fsspec/registry.py:290: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify

Max number of rows allowed is: 1000
Total number of keys: 13131687
Auto sample fraction: 7.615167799841712e-05
parsing model-builder asset


ArrowInvalid: No match for FieldRef.Name(trade_hcAmt_accts_never_dq__max_by_acctType_utility) in trade_acctType__count: float
trade_acctType__nunique: float
trade_blncAmt__max: float
trade_blncAmt__max_by_acctType_auto: float
trade_blncAmt__max_by_acctType_charge_card: float
trade_blncAmt__max_by_acctType_credit_card: float
trade_blncAmt__max_by_acctType_education: float
trade_blncAmt__max_by_acctType_secure: float
trade_blncAmt__max_by_acctType_unsecure: float
trade_blncAmt__max_by_ecoa_authorized: float
trade_blncAmt__max_by_ecoa_individual: float
trade_blncAmt__max_by_ecoa_joint: float
trade_blncAmt__max_by_prtfType_installment: float
trade_blncAmt__max_by_prtfType_mortgage: float
trade_blncAmt__max_by_prtfType_revolving: float
trade_blncAmt__mean: float
trade_blncAmt__mean_by_acctType_auto: float
trade_blncAmt__mean_by_acctType_charge_card: float
trade_blncAmt__mean_by_acctType_credit_card: float
trade_blncAmt__mean_by_acctType_education: float
trade_blncAmt__mean_by_acctType_secur

In [33]:
temp_equifax = pd.read_parquet('s3://power-equifax-prod/NATIONAL/PROCESSED/DATA/BUREAU=equifax/FORMAT=cms_6/TABLE=trade_fe/PULL_NAME=national_2/FEW_VERSION=0.1.0/ME_VERSION=v1.10.2/DATE_OF_REQUEST=2020-06-30/ACCOUNT_TYPE_CODE=3A/GEO_STATE_CODE=LA/a26b42f3562749e2baad25b01cbb459e-0.parquet')
temp_equifax.shape

(414, 8464)

In [29]:
temp_tu4r = pd.read_parquet('s3://power-transunion-prod/NATIONAL/PROCESSED/DATA/BUREAU=transunion/FORMAT=TU4R/TABLE=trade_fe/PULL_NAME=national_4/FEW_VERSION=0.1.0/ME_VERSION=v1.10.2/DATE_OF_REQUEST=2020-06-30/ACCOUNT_TYPE_CODE=AU/GEO_STATE_CODE=VT/9d535aab53a34c699d44575d4e076983-0.parquet')
temp_tu4r.shape

(11415, 8464)

In [34]:
'trade_hcAmt_accts_never_dq__max_by_acctType_utility' in temp_equifax.columns

True

In [30]:
'trade_hcAmt_accts_never_dq__max_by_acctType_utility' in temp_tu4r.columns

True

In [ ]:
scoreFiles = []

for j in ['_auc.json','_ks.json']:
    for i in ['train', 'test']:
        scoreFiles.append(i+j)
        
scoreFiles_model = {}
for i in scoreFiles:
    name = i.split('.')[0]
    # print(name)
    scoreFiles_model[name] = json.load(open(os.path.join(national_model_output_path, i), 'rb'))


for i in scoreFiles_model:
    print(i, scoreFiles_model[i])

In [15]:
scoreFiles = []

for j in ['_auc.json','_ks.json']:
    for i in ['train', 'test']:
        scoreFiles.append(i+j)
        
scoreFiles_model = {}
for i in scoreFiles:
    name = i.split('.')[0]
    # print(name)
    scoreFiles_model[name] = json.load(open(os.path.join(feature_selection_model_output_path, i), 'rb'))


for i in scoreFiles_model:
    print(i, scoreFiles_model[i])

train_auc {'auc': 0.8686551218870335}
test_auc {'auc': 0.8535502746706296}
train_ks {'ks': 0.5812319389933264}
test_ks {'ks': 0.5624095655404407}


In [20]:
ftr_importance = pd.read_parquet('/d/shared/users/lyt/penfed_poc1_national_models/nationalmodel1a/feature_importance.parquet')
ftr_importance.head(10)

,feature,importance
0,trade_clAmt_accts_with_rcntPmt__mean_by_prtfTy...,0.003050
1,trade_clAmt_accts_never_dq__sum_by_prtfType_re...,0.003048
2,inq_inquiries__count_by_indCode_auto_mortgage,0.003005
3,inq_inquiries_in_last_year__sum,0.002900
4,trade_clAmt_active_accts__mean_by_prtfType_rev...,0.002854
5,inq_inquiries__sum,0.002753
6,trade_hcAmt__min_by_prtfType_installment,0.002750
7,trade_blncAmt__mean_by_acctType_auto,0.002422
8,trade_hcAmt_accts_never_dq__sum,0.002365
9,trade_hcAmt_all_non_derog_accts__sum,0.002268


In [21]:
ftr_importance[ftr_importance.feature == 'trade_hcAmt_accts_never_dq__max_by_acctType_utility']

,feature,importance
277,trade_hcAmt_accts_never_dq__max_by_acctType_ut...,0.000139
